In [3]:
# Import libraries
%load_ext autoreload
%autoreload 2

from pulse_lib.pulse_program import *
import pickle

from qick import *
import Pyro4
%pylab inline

# Pyro communicates with board
Pyro4.config.SERIALIZER = "pickle"
Pyro4.config.PICKLE_PROTOCOL_VERSION=4

# Static IP proxy
ns_host = "192.168.2.99"
ns_port = 8888
proxy_name = "myqick"

# QICK object
ns = Pyro4.locateNS(host=ns_host, port=ns_port)
soc = Pyro4.Proxy(ns.lookup(proxy_name))
soccfg = QickConfig(soc.get_cfg())
print(soccfg)

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib

QICK configuration:

	Board: RFSoC4x2

	Software version: 0.2.267
	Firmware timestamp: Wed Sep  6 18:49:29 2023

	Global clocks (MHz): tProcessor 409.600, RF reference 491.520

	2 signal generator channels:
	0:	axis_signal_gen_v6 - envelope memory 65536 samples (6.667 us)
		fs=9830.400 MHz, fabric=614.400 MHz, 32-bit DDS, range=9830.400 MHz
		DAC tile 0, blk 0 is DAC_B
	1:	axis_signal_gen_v6 - envelope memory 65536 samples (6.667 us)
		fs=9830.400 MHz, fabric=614.400 MHz, 32-bit DDS, range=9830.400 MHz
		DAC tile 2, blk 0 is DAC_A

	2 readout channels:
	0:	axis_readout_v2 - configured by PYNQ
		fs=4423.680 MHz, decimated=552.960 MHz, 32-bit DDS, range=4423.680 MHz
		maxlen 16384 accumulated, 1024 decimated (1.852 us)
		triggered by output 7, pin 14, feedback to tProc input 0
		ADC tile 0, blk 0 is ADC_D
	1:	axis_readout_v2 - configured by PYNQ


In [4]:
prog = QickProgram(soccfg)

nums = [5e-3, 6e-3]
for num in nums:
    cycles = prog.us2cycles(num, gen_ch=0)
    print(cycles)
    us = prog.cycles2us(cycles, gen_ch=0)
    print(us*1e3)

3
4.8828125
4
6.510416666666667


# Shortest Pulse Length

- [ ] Add to pulse program?

In [18]:
pulse_resolution = prog.cycles2us(1, gen_ch=0) * 1e3
min_pulse_len = prog.cycles2us(3, gen_ch=0) * 1e3
max_pulse_len = prog.cycles2us(65535, gen_ch=0) * 1e3
pulse_freq = 1/prog.cycles2us(1, gen_ch=0)

print(f"Pulse length sampling frequency: {pulse_freq} [MHz]")
print(f"--> Shortest pulse length (2^2 - 1 cycles): {round(min_pulse_len,2)} [ns]")
print(f"--> Longest pulse length (2^16 - 1 cycles): {round(max_pulse_len,2)} [ns]")
print(pulse_resolution)

Pulse length sampling frequency: 614.4 [MHz]
--> Shortest pulse length (2^2 - 1 cycles): 4.88 [ns]
--> Longest pulse length (2^16 - 1 cycles): 106665.04 [ns]
1.6276041666666667


In [22]:
start_resolution = prog.cycles2us(1) * 1e3
print(start_resolution)

2.44140625
